In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Main

In [2]:
! pip install Cython==0.29.36
! pip install fasttext==0.8.3
! pip install kenlm
! pip install torchtext==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.0 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.8.3-cp310-cp310-linux_x86_64.whl size=1481037 sha256=0813cbe30963d5091af6273fd27cf2b4b78fcaff610d7ba0ede686fc5bc66ccb
  Stored in directory: /root/.cache/pip/wheels/2b/85/8b/3037ff2b0c1297bd87e6deee5c9a06e4b4da5f4c448c66f5f5
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184433 sha256=399e65c12b7f062af4f49d0578b45f6d

In [3]:
%cd /content/gdrive/MyDrive/CSCI 544/project/style-transformer

/content/gdrive/MyDrive/CSCI 544/project/style-transformer


In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch
import time
from data import load_dataset
from models import StyleTransformer, Discriminator
from train import train, auto_eval
from torchsummary import summary

class Config():
    data_path = './data/amazon35/'
    log_dir = 'runs/exp'
    dataName = 'amazon35'
    save_path = './save'
    pretrained_embed_path = './embedding/'
    device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')
    discriminator_method = 'Cond' # 'Multi' or 'Cond'
    load_pretrained_embed = True
    min_freq = 3
    max_length = 36
    embed_size = 200
    d_model = 200
    h = 4
    num_styles = 2
    num_classes = num_styles + 1 if discriminator_method == 'Multi' else 2
    num_layers = 4
    batch_size = 64
    lr_F = 0.0001
    lr_D = 0.0001
    L2 = 0
    iter_D = 10
    iter_F = 5
    F_pretrain_iter = 800
    log_steps = 5
    eval_steps = 25
    learned_pos_embed = True
    dropout = 0
    drop_rate_config = [(1, 0)]
    temperature_config = [(1, 0)]

    slf_factor = 0.25
    cyc_factor = 0.5
    adv_factor = 1

    inp_shuffle_len = 0
    inp_unk_drop_fac = 0
    inp_rand_drop_fac = 0
    inp_drop_prob = 0

def main():
    config = Config()
    train_iters, dev_iters, test_iters, vocab = load_dataset(config)
    print('Vocab size:', len(vocab))
    model_F = StyleTransformer(config, vocab)
    model_D = Discriminator(config, vocab)
    testmodel_F = StyleTransformer(config, vocab)

    pretrained_F_dict = torch.load('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/yelp_glove200_F.pth', map_location=torch.device('cpu'))
    pretrained_D_dict = torch.load('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/yelp_glove200_D.pth', map_location=torch.device('cpu'))

    #  load generator - yelp pretrain model
    model_F_dict = model_F.state_dict()
    pretrained_F_dict = {k: v for k, v in pretrained_F_dict.items() if 'generator' not in k and 'embed' not in k}
    model_F_dict.update(pretrained_F_dict)
    model_F.load_state_dict(model_F_dict)

    model_F.encoder.requires_grad_(False)
    model_F.decoder.requires_grad_(False)
    model_F.decoder.generator.requires_grad_(True)

    model_F.to(config.device)

    model_D_dict = model_D.state_dict()
    pretrained_D_dict = {k: v for k, v in pretrained_D_dict.items() if k.startswith('embed') == False}
    model_D_dict.update(pretrained_D_dict)
    model_D.load_state_dict(model_D_dict)

    model_D.encoder.requires_grad_(False)
    model_D.to(config.device)

    # for name, param in model_F.named_parameters() :
    #   print(name, param.requires_grad)

    print(config.discriminator_method)

    train(config, vocab, model_F, model_D, train_iters, dev_iters, test_iters)


if __name__ == '__main__':
    main()

vectors torch.Size([21760, 200])
load embedding took 1.29 s.
Vocab size: 21760
embed loaded.
embed loaded.
embed loaded.
sos_token True
style_embed.weight True
embed.token_embed.weight False
embed.pos_embed.weight True
encoder.layers.0.self_attn.head_projs.0.weight False
encoder.layers.0.self_attn.head_projs.0.bias False
encoder.layers.0.self_attn.head_projs.1.weight False
encoder.layers.0.self_attn.head_projs.1.bias False
encoder.layers.0.self_attn.head_projs.2.weight False
encoder.layers.0.self_attn.head_projs.2.bias False
encoder.layers.0.self_attn.fc.weight False
encoder.layers.0.self_attn.fc.bias False
encoder.layers.0.pw_ffn.mlp.0.weight False
encoder.layers.0.pw_ffn.mlp.0.bias False
encoder.layers.0.pw_ffn.mlp.3.weight False
encoder.layers.0.pw_ffn.mlp.3.bias False
encoder.layers.0.sublayer.0.layer_norm.weight False
encoder.layers.0.sublayer.0.layer_norm.bias False
encoder.layers.0.sublayer.1.layer_norm.weight False
encoder.layers.0.sublayer.1.layer_norm.bias False
encoder.layer

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

******************** neg sample ********************
[gold] my brother printer does not detect these cartridges ...need to return .
[raw ] my printer did not to use .
[rev ] my printer did not to use .
[ref ] expensive the real price should not be more than 

******************** neg sample ********************
[gold] lasted around days until the keyring section irreparably from the coil .
[raw ] love this is a little product and i have to be much of the price .
[rev ] love this is a little product and i have to be much of the price .
[ref ] smells weird

******************** neg sample ********************
[gold] very poorly refilled toner cartridges .
[raw ] very nice product and it is a great for the price .
[rev ] very nice product and it is a great for the price .
[ref ] not reliable

******************** neg sample ********************
[gold] poor quality materials .
[raw ] excellent product .
[rev ] excellent product .
[ref ] this desk is only worth about bucks do not buy

*****

AttributeError: ignored

# **Retrained hw1 data**

In [4]:
! pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [6]:
train_list = []
train_number = 0
test_number = 0
with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.pos', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 1])
        train_number += 1

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.neg', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 0])
        train_number += 1

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.pos', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 1])
        test_number += 1

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.neg', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 0])
        test_number += 1

In [7]:
print("train number", train_number, " test number", test_number)

train number 504976  test number 1000


In [8]:
df = pd.DataFrame(train_list, columns = ['texts', 'rate'])
df.head()

,texts,rate
0,was a little crooked when i received it but it...,1
1,this is a great way to freshen up a workstatio...,1
2,quality product,1
3,finally hit pay dirt . after searching i final...,1
4,excellent quality and exactly what i was needi...,1


In [9]:
df['texts'] = [re.sub(r'[^a-zA-Z]',' ',str(x)) for x in df['texts']]

In [10]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [11]:
df['texts'] = df['texts'].apply(lambda x : [word for word in x.split(' ') if word not in stop_words])
df['texts'] = df['texts'].apply(lambda x : [word for word in x if word != ''])

In [12]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmantizer = WordNetLemmatizer()

In [13]:
def get_wordnet_pos(word) :
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

df['texts'] = df['texts'].apply(lambda x : [lemmantizer.lemmatize(word,get_wordnet_pos(word)) for word in x])

In [14]:
df['texts'] = df['texts'].apply(lambda x : " ".join(x))
text_data = [''.join(tokens).strip() for tokens in df['texts']]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=700)
bow_matrix = bow.fit_transform(text_data).toarray()

bow_df = pd.DataFrame(bow_matrix, columns=bow.get_feature_names_out())

In [17]:
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support

train_x = bow_df[:504976]
train_y = df.loc[:504975,['rate']]
test_x = bow_df[504976:]
test_y = df.loc[504976:, ['rate']]

print(len(train_x), ' ', len(train_y))
clf = svm.LinearSVC(max_iter=3000)
clf.fit(train_x, train_y)

y_pred = clf.predict(test_x)
precision_recall_fscore_support(test_y, y_pred, average='macro')

504976   504976


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


(0.8373819163292847, 0.8360000000000001, 0.8358318918572618, None)

# **Discriminator**
First, we train the model in the paper using amazon dataset35. And then we use this pretrained discriminator to do the comparison with hw1


In [21]:
from train import get_lengths, batch_preprocess
def predict(config, vocab, model_D, test_iters) :
  model_D.eval()

  batch_iters = iter(test_iters)
  prediction = []
  labels = []

  for i in range(0, 8) :
    batch = next(batch_iters)

    pad_idx = vocab.stoi['<pad>']
    eos_idx = vocab.stoi['<eos>']
    vocab_size = len(vocab)
    inp_tokens, inp_lengths, raw_styles = batch_preprocess(batch, pad_idx, eos_idx)
    # print(inp_tokens.size())
    rev_styles = 1 - raw_styles

    raw_gold_log_probs = model_D(inp_tokens, inp_lengths)
    pred = torch.argmax(raw_gold_log_probs, dim=1)
    # print(inp_tokens)
    # print(raw_styles)
    pred = pred.tolist()

    prediction = prediction + pred
    labels = labels + raw_styles.tolist()

  print(precision_recall_fscore_support(prediction, labels, average='macro'))

In [20]:
import torch
import time
from data import load_dataset
from models import StyleTransformer, Discriminator
from train import train, auto_eval
from torchsummary import summary

class Config():
    data_path = './data/amazon35/'
    log_dir = 'runs/exp'
    dataName = 'amazon35'
    pretrained_embed_path = './embedding/'
    device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')
    discriminator_method = 'Cond' # 'Multi' or 'Cond'
    load_pretrained_embed = True
    min_freq = 3
    max_length = 36
    embed_size = 200
    d_model = 200
    h = 4
    num_styles = 2
    num_classes = num_styles + 1 if discriminator_method == 'Multi' else 2
    num_layers = 4
    batch_size = 64
    lr_F = 0.0001
    lr_D = 0.0001
    L2 = 0
    iter_D = 10
    iter_F = 5
    F_pretrain_iter = 800
    log_steps = 5
    eval_steps = 25
    learned_pos_embed = True
    dropout = 0
    drop_rate_config = [(1, 0)]
    temperature_config = [(1, 0)]

    slf_factor = 0.25
    cyc_factor = 0.5
    adv_factor = 1

    inp_shuffle_len = 0
    inp_unk_drop_fac = 0
    inp_rand_drop_fac = 0
    inp_drop_prob = 0

config = Config()
train_iters, dev_iters, test_iters, vocab = load_dataset(config)
print('Vocab size:', len(vocab))

model_D = Discriminator(config, vocab)

pretrained_D_dict = torch.load('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/2300_D.pth')

model_D.load_state_dict(pretrained_D_dict)
model_D.to(config.device)
print(config.discriminator_method)

# predict(config, vocab, model_D, test_iters)

vectors torch.Size([21760, 200])
load embedding took 3.65 s.
Vocab size: 21760
embed loaded.
Cond


In [22]:
from sklearn.metrics import precision_recall_fscore_support
predict(config, vocab, model_D, test_iters)

(0.914, 0.9141060111388516, 0.9139944956477215, None)
